In [71]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

**Логистическая регрессия**

Логистическая регрессия используется для задач классификации, при этом вычисляется вероятность принадлежности события к определенному классу.

Задача алгоритма логистической регрессии - найти подходящие коэффициенты w при признаках x:

y = w<sub>0</sub> + w<sub>1</sub> * x<sub>1</sub> + w<sub>2</sub> * x<sub>2</sub> + ... + w<sub>m</sub> * x<sub>m</sub>

Величина z помещается в сигмоиндную функцию для вычисления вероятности:

f(x) = 1 / (1 + e<sup>-z</sup>)

Значение f(x) будет расположено на отрезке [0, 1].

f(x) - вероятность отнесения события к классу 1,

1 - f(x) - вероятность отнесения события к классу 0.

**Log loss**

Алгоритм логистической регрессии минимизирует величину logloss:

logloss = -y * ln(p) - (1 - y) * ln(1 - p)

где y - истинное значение (0 или 1), 

p - вычисленная алгоритмом вероятность того, что событие принадлежит к классу 1.

Чем ниже logloss, тем более точные ответы мы получаем используя логистическую регрессию.

**Titanic**

датасет: https://www.kaggle.com/c/titanic/data?select=train.csv

In [72]:
data = pd.read_csv('train.csv')

In [73]:
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Установим в качестве индекса PassengerId - будет проще отслеживать, с информацией о каком пассажире мы работаем

In [74]:
data = data.set_index('PassengerId')

In [75]:
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [76]:
data.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')

Наша задача - определить, выжил ли человек при крушении Титаника.

В поле Survived выжившие 1, утонувшие - 0.

In [77]:
target = 'Survived'

In [78]:
y = data[target]
y

PassengerId
1      0
2      1
3      1
4      1
5      0
      ..
887    0
888    1
889    0
890    1
891    0
Name: Survived, Length: 891, dtype: int64

In [79]:
X = data.drop(target, axis=1)
X

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...
887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


**Изучение качества данных и их очистка**

In [80]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Cabin     204 non-null    object 
 9   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 76.6+ KB


В поле Cabin слишком много пропущенных значений, не будем его рассматривать.

Столбцы Name и Ticket также не будем.

Удалим эти поля.

In [81]:
X = X.drop(['Cabin', 'Name', 'Ticket'], axis=1)
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,male,22.0,1,0,7.2500,S
2,1,female,38.0,1,0,71.2833,C
3,3,female,26.0,0,0,7.9250,S
4,1,female,35.0,1,0,53.1000,S
5,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...
887,2,male,27.0,0,0,13.0000,S
888,1,female,19.0,0,0,30.0000,S
889,3,female,NaN,1,2,23.4500,S


В поле возраст 20% значений не заполнено. Заменим пропущенные значения на значения среднего возраста.

In [82]:
mean_age = X['Age'].mean()
mean_age

np.float64(29.69911764705882)

In [83]:
X['Age'] = X['Age'].fillna(mean_age)

In [84]:
X['Age'].unique()

array([22.        , 38.        , 26.        , 35.        , 29.69911765,
       54.        ,  2.        , 27.        , 14.        ,  4.        ,
       58.        , 20.        , 39.        , 55.        , 31.        ,
       34.        , 15.        , 28.        ,  8.        , 19.        ,
       40.        , 66.        , 42.        , 21.        , 18.        ,
        3.        ,  7.        , 49.        , 29.        , 65.        ,
       28.5       ,  5.        , 11.        , 45.        , 17.        ,
       32.        , 16.        , 25.        ,  0.83      , 30.        ,
       33.        , 23.        , 24.        , 46.        , 59.        ,
       71.        , 37.        , 47.        , 14.5       , 70.5       ,
       32.5       , 12.        ,  9.        , 36.5       , 51.        ,
       55.5       , 40.5       , 44.        ,  1.        , 61.        ,
       56.        , 50.        , 36.        , 45.5       , 20.5       ,
       62.        , 41.        , 52.        , 63.        , 23.5 

В поле Embarked, которое является текстовым, не хватает двух значений, заполним пропущенные значения наиболее часто встречающимся значением (мода)

In [85]:
X['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [86]:
embarked_mode = X['Embarked'].mode()[0]
embarked_mode

'S'

In [87]:
X['Embarked'] = X['Embarked'].fillna(embarked_mode)

In [88]:
X['Embarked'].value_counts()

Embarked
S    646
C    168
Q     77
Name: count, dtype: int64

In [89]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 55.7+ KB


**Работа с категориальными признаками**

Признак пола пассажира (Sex) - категориальная переменная. Так как в ней два класса, ты мы можем представить одной колонкой со значениями 0 и 1:

In [90]:
X['Sex'].value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [91]:
X['Sex'] = (X['Sex'] == 'female').astype(int)
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1,3,0,22.000000,1,0,7.2500,S
2,1,1,38.000000,1,0,71.2833,C
3,3,1,26.000000,0,0,7.9250,S
4,1,1,35.000000,1,0,53.1000,S
5,3,0,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...
887,2,0,27.000000,0,0,13.0000,S
888,1,1,19.000000,0,0,30.0000,S
889,3,1,29.699118,1,2,23.4500,S


In [92]:
X['Sex'].value_counts()

Sex
0    577
1    314
Name: count, dtype: int64

Еще одна категориальная переменная - Embarked.

Преобразуем ее значения в dummy-переменные.

Теперь каждому ее значению будет отведен отдельный столбец, в котором возможны два числа:

1 (переменная равна данному значению), 

0 (переменная не равна данному значению)

In [93]:
X = pd.get_dummies(X)
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
1,3,0,22.000000,1,0,7.2500,False,False,True
2,1,1,38.000000,1,0,71.2833,True,False,False
3,3,1,26.000000,0,0,7.9250,False,False,True
4,1,1,35.000000,1,0,53.1000,False,False,True
5,3,0,35.000000,0,0,8.0500,False,False,True
...,...,...,...,...,...,...,...,...,...
887,2,0,27.000000,0,0,13.0000,False,False,True
888,1,1,19.000000,0,0,30.0000,False,False,True
889,3,1,29.699118,1,2,23.4500,False,False,True


Переменная Embarked преобразовалась в дамми-переменные Embarked_C, Embarked_Q, Embarked_S

In [94]:
X.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_C',
       'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [95]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Sex         891 non-null    int64  
 2   Age         891 non-null    float64
 3   SibSp       891 non-null    int64  
 4   Parch       891 non-null    int64  
 5   Fare        891 non-null    float64
 6   Embarked_C  891 non-null    bool   
 7   Embarked_Q  891 non-null    bool   
 8   Embarked_S  891 non-null    bool   
dtypes: bool(3), float64(2), int64(4)
memory usage: 51.3 KB


In [96]:
X[['Embarked_C', 'Embarked_Q', 'Embarked_S']] = X[['Embarked_C', 'Embarked_Q', 'Embarked_S']].astype(int)
X

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
1,3,0,22.000000,1,0,7.2500,0,0,1
2,1,1,38.000000,1,0,71.2833,1,0,0
3,3,1,26.000000,0,0,7.9250,0,0,1
4,1,1,35.000000,1,0,53.1000,0,0,1
5,3,0,35.000000,0,0,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...
887,2,0,27.000000,0,0,13.0000,0,0,1
888,1,1,19.000000,0,0,30.0000,0,0,1
889,3,1,29.699118,1,2,23.4500,0,0,1


In [97]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      891 non-null    int64  
 1   Sex         891 non-null    int64  
 2   Age         891 non-null    float64
 3   SibSp       891 non-null    int64  
 4   Parch       891 non-null    int64  
 5   Fare        891 non-null    float64
 6   Embarked_C  891 non-null    int64  
 7   Embarked_Q  891 non-null    int64  
 8   Embarked_S  891 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 69.6 KB


In [98]:
X.head(10)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,
1,3,0,22.000000,1,0,7.2500,0,0,1
2,1,1,38.000000,1,0,71.2833,1,0,0
3,3,1,26.000000,0,0,7.9250,0,0,1
4,1,1,35.000000,1,0,53.1000,0,0,1
5,3,0,35.000000,0,0,8.0500,0,0,1
6,3,0,29.699118,0,0,8.4583,0,1,0
7,1,0,54.000000,0,0,51.8625,0,0,1
8,3,0,2.000000,3,1,21.0750,0,0,1
9,3,1,27.000000,0,2,11.1333,0,0,1


Переменную Pclass можно отправить в модель в таком виде, какая она есть и сейчас, так как она представлена числами, но так как это все-таки не 

количественная переменная (например, нельзя сказать, что между классами 1 и 2 такая же "разность", как и между классами 2 и 3), то ее так же 

переведем в дамми-переменные. Для этого поменяем ее тип данных на category и применим еще раз функцию get_dummies ко всему датафрейму.

Эта функция переведет в дамми-переменные все нечисловые признаки.


In [99]:
X['Pclass'].unique()

array([3, 1, 2])

In [100]:
X['Pclass'] = X['Pclass'].astype('category')
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   Pclass      891 non-null    category
 1   Sex         891 non-null    int64   
 2   Age         891 non-null    float64 
 3   SibSp       891 non-null    int64   
 4   Parch       891 non-null    int64   
 5   Fare        891 non-null    float64 
 6   Embarked_C  891 non-null    int64   
 7   Embarked_Q  891 non-null    int64   
 8   Embarked_S  891 non-null    int64   
dtypes: category(1), float64(2), int64(6)
memory usage: 63.6 KB


In [101]:
X = pd.get_dummies(X)
X

,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,,
1,0,22.000000,1,0,7.2500,0,0,1,False,False,True
2,1,38.000000,1,0,71.2833,1,0,0,True,False,False
3,1,26.000000,0,0,7.9250,0,0,1,False,False,True
4,1,35.000000,1,0,53.1000,0,0,1,True,False,False
5,0,35.000000,0,0,8.0500,0,0,1,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
887,0,27.000000,0,0,13.0000,0,0,1,False,True,False
888,1,19.000000,0,0,30.0000,0,0,1,True,False,False
889,1,29.699118,1,2,23.4500,0,0,1,False,False,True


In [102]:
X[['Pclass_1', 'Pclass_2', 'Pclass_3']] = X[['Pclass_1', 'Pclass_2', 'Pclass_3']].astype(int)
X

,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,,
1,0,22.000000,1,0,7.2500,0,0,1,0,0,1
2,1,38.000000,1,0,71.2833,1,0,0,1,0,0
3,1,26.000000,0,0,7.9250,0,0,1,0,0,1
4,1,35.000000,1,0,53.1000,0,0,1,1,0,0
5,0,35.000000,0,0,8.0500,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
887,0,27.000000,0,0,13.0000,0,0,1,0,1,0
888,1,19.000000,0,0,30.0000,0,0,1,1,0,0
889,1,29.699118,1,2,23.4500,0,0,1,0,0,1


In [103]:
X.columns

Index(['Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_C', 'Embarked_Q',
       'Embarked_S', 'Pclass_1', 'Pclass_2', 'Pclass_3'],
      dtype='object')

In [104]:
X.head(10)

,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,,
1,0,22.000000,1,0,7.2500,0,0,1,0,0,1
2,1,38.000000,1,0,71.2833,1,0,0,1,0,0
3,1,26.000000,0,0,7.9250,0,0,1,0,0,1
4,1,35.000000,1,0,53.1000,0,0,1,1,0,0
5,0,35.000000,0,0,8.0500,0,0,1,0,0,1
6,0,29.699118,0,0,8.4583,0,1,0,0,0,1
7,0,54.000000,0,0,51.8625,0,0,1,1,0,0
8,0,2.000000,3,1,21.0750,0,0,1,0,0,1
9,1,27.000000,0,2,11.1333,0,0,1,0,0,1


**Цель разбиения данных на тренировочный, валидационный и тестовый датасеты**

В этот раз мы будем работать с разделением данных для модели более тщательно, мы разобьем данные из файла train.csv на две части:

тренировочный набор данных (признаки в X_train, целевая переменная в y_train) и валидационный (соответственно X_valid и y_valid).

На тренировочном датасете мы будем строить модель, а на валидационном - проверять ее качество. После проверки на валидационном сете

можно будет окончательно проверить модель на тестовом сете (содержится в файле test.csv по вышеуказанной ссылке - фактическое 

значение целевой переменной в тестовом наборе отсутствует, рекомендуется самостоятельно проверить точность данных при отправке

решения на странице соревнования).

In [105]:
X

,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,,
1,0,22.000000,1,0,7.2500,0,0,1,0,0,1
2,1,38.000000,1,0,71.2833,1,0,0,1,0,0
3,1,26.000000,0,0,7.9250,0,0,1,0,0,1
4,1,35.000000,1,0,53.1000,0,0,1,1,0,0
5,0,35.000000,0,0,8.0500,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
887,0,27.000000,0,0,13.0000,0,0,1,0,1,0
888,1,19.000000,0,0,30.0000,0,0,1,1,0,0
889,1,29.699118,1,2,23.4500,0,0,1,0,0,1


In [107]:
# Параметр test_size будет определять, какую долю данных из train.csv мы берем для валидационного датасета
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

**Построение модели**

В начале работы над любой задачей рекомендуется сначала сделать простую модель, не затрачивая большого количества времени.

Затем можно будет оценивать новые модели, сравнивая их качество с качеством первоначальной (базовой) модели.

Сначала посмотрим на информацию о признаках:

In [110]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 668 entries, 299 to 103
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Sex         668 non-null    int64  
 1   Age         668 non-null    float64
 2   SibSp       668 non-null    int64  
 3   Parch       668 non-null    int64  
 4   Fare        668 non-null    float64
 5   Embarked_C  668 non-null    int64  
 6   Embarked_Q  668 non-null    int64  
 7   Embarked_S  668 non-null    int64  
 8   Pclass_1    668 non-null    int64  
 9   Pclass_2    668 non-null    int64  
 10  Pclass_3    668 non-null    int64  
dtypes: float64(2), int64(9)
memory usage: 62.6 KB


Все признаки - числовые, поэтому мы можем построить модель, используя все признаки.

In [111]:
lr = LogisticRegression()

In [112]:
lr.fit(X_train, y_train)

LogisticRegression()

In [113]:
y_pred = lr.predict(X_valid)

In [114]:
y_pred

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0])

Метрика Accuracy (доля правильных ответов)

In [115]:
from sklearn.metrics import accuracy_score

In [116]:
accuracy_score(y_valid, y_pred)

0.8026905829596412

Можно сравнить с Accuracy на тренировочном датасете

In [117]:
y_pred_train = lr.predict(X_train)

In [118]:
accuracy_score(y_train, y_pred_train)

0.8083832335329342

**Вычисление вероятностей событий**

In [119]:
y_proba = lr.predict_proba(X_valid)

In [121]:
#Вероятности событий 0 и 1 для каждого пасссажира
y_proba

array([[0.88831594, 0.11168406],
       [0.73067479, 0.26932521],
       [0.87040428, 0.12959572],
       [0.08976523, 0.91023477],
       [0.2512522 , 0.7487478 ],
       [0.07820807, 0.92179193],
       [0.33314033, 0.66685967],
       [0.90547214, 0.09452786],
       [0.24431507, 0.75568493],
       [0.10305508, 0.89694492],
       [0.69433176, 0.30566824],
       [0.93500822, 0.06499178],
       [0.62516845, 0.37483155],
       [0.84808965, 0.15191035],
       [0.75857584, 0.24142416],
       [0.07880567, 0.92119433],
       [0.72748315, 0.27251685],
       [0.33305598, 0.66694402],
       [0.70245252, 0.29754748],
       [0.70644471, 0.29355529],
       [0.88408923, 0.11591077],
       [0.6427338 , 0.3572662 ],
       [0.39866422, 0.60133578],
       [0.86969962, 0.13030038],
       [0.89854001, 0.10145999],
       [0.92825944, 0.07174056],
       [0.56620371, 0.43379629],
       [0.72830965, 0.27169035],
       [0.91448475, 0.08551525],
       [0.42676834, 0.57323166],
       [0.

Нас интересует вероятность события 1 (правый столбец) - при получении ответа значения из него округляются до 0 или до 1

**Сохранение данных**

In [122]:
X_train.to_pickle('X_train.pkl')
y_train.to_pickle('y_train.pkl')

X_valid.to_pickle('X_valid.pkl')
y_valid.to_pickle('y_valid.pkl')